## Predict the number of abalone's hook
**How deep learning framework is not used**

### Import modules  
**numpy** : Support the array and numerical computation  
**csv** : read the Abalone dataset  
**time** : Initialization of random functions  

In [1]:
import numpy as np
import csv
import time

np.random.seed(1234)
def randomize(): np.random.seed(time.time())

### Definition of the hyper parameters  
The following three are the mean, standard deviation of the normalized distribution and a learning rate. 

In [2]:
RND_MEAN = 0
RND_STD = 0.0030

LEARNING_RATE = 0.001

### Main function for the experiment  
Create deep learning model by calling the functions below in turn. Deal with the entire process.

In [3]:
def abalone_exec(epoch_count=10, mb_size=10, report=1):
    load_abalone_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

In [4]:
def load_abalone_dataset():
    with open('./abalone.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        for row in csvreader:
            rows.append(row)
            
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 10, 1
    data = np.zeros([len(rows), input_cnt+output_cnt])

    for n, row in enumerate(rows):
        if row[0] == 'I': 
            data[n, 0] = 1
        if row[0] == 'M': 
            data[n, 1] = 1
        if row[0] == 'F': 
            data[n, 2] = 1
        data[n, 3:] = row[1:]

In [5]:
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
    bias = np.zeros([output_cnt])

In [6]:
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()

    for epoch in range(epoch_count):
        losses, accs = [], []

        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)

        if report > 0 and (epoch + 1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'.\
                format(epoch + 1, np.mean(losses), np.mean(accs), acc))

    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.
          format(final_acc))

In [7]:
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count

In [8]:
def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt]

In [9]:
def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size * nth: mb_size * (nth + 1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

In [10]:
def run_train(x, y):
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)

    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)

    return loss, accuracy

In [11]:
def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

In [12]:
def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias
    return output, x

In [13]:
def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose()

    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis=0)

    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

In [14]:
def forward_postproc(output, y):
    diff = output - y
    square = np.square(diff)
    loss = np.mean(square)
    return loss, diff

In [15]:
def backprop_postproc(G_loss, diff):
    shape = diff.shape

    g_loss_square = np.ones(shape) / np.prod(shape)
    g_square_diff = 2 * diff
    g_diff_output = 1

    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff

    return G_output

In [16]:
def backprop_postproc_oneline(G_loss, diff):
    return 2 * diff / np.prod(diff.shape)

In [17]:
def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output - y) / y))
    return 1 - mdiff

In [18]:
if __name__ == "__main__":
    abalone_exec()

Epoch 1: loss=33.707, accuracy=0.554/0.693
Epoch 2: loss=8.238, accuracy=0.818/0.674
Epoch 3: loss=7.597, accuracy=0.810/0.674
Epoch 4: loss=7.487, accuracy=0.809/0.674
Epoch 5: loss=7.405, accuracy=0.808/0.676
Epoch 6: loss=7.335, accuracy=0.808/0.679
Epoch 7: loss=7.272, accuracy=0.809/0.678
Epoch 8: loss=7.222, accuracy=0.807/0.681
Epoch 9: loss=7.175, accuracy=0.808/0.683
Epoch 10: loss=7.133, accuracy=0.809/0.683

Final Test: final accuracy = 0.683
